In [25]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ['OCTOAI_API_TOKEN']
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

url = "https://www.microsoft.com/en-us/corporate-responsibility/sustainability-journey"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("div", "Divider")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

In [27]:
chunk_size = 1024
chunk_overlap = 128
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# Split
splits = text_splitter.split_documents(html_header_splits)

In [28]:
from langchain.vectorstores import FAISS

In [29]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="llama-2-13b-chat-fp16",
        max_tokens=1024,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
        
    )
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1")

c:\Users\sasgi\source\sustAInable\test\Lib\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [30]:
vector_store = FAISS.from_documents(
    splits,
    embedding=embeddings
)

In [31]:
retriever = vector_store.as_retriever()

In [32]:
from langchain.prompts import ChatPromptTemplate
template="""You are a sustainability report writer. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [33]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
text = chain.invoke("Write a report to the SEC that discloses information about the registrant’s climate-related targets or goals, if any, that have materially affected or are reasonably likely to materially affect the registrant's business, results of operations, or financial condition. Disclosures would include material expenditures and material impacts on financial estimates and assumptions as a direct result of the target or goal or actions taken to make progress toward meeting such target or goal.")

print(text)

 
As a sustainability report writer, I am pleased to provide the following report to the SEC regarding Microsoft's climate-related targets or goals that have materially affected or are reasonably likely to materially affect the registrant's business, results of operations, or financial condition.

Microsoft has set a goal to become carbon negative by 2030, which means that the company will remove more carbon dioxide from the atmosphere than it emits. This goal has been a driving force behind the company's sustainability efforts and has led to significant investments in renewable energy, energy efficiency, and carbon capture and storage.

In 2022, Microsoft made a commitment to power 50% of its data centers with renewable energy by 2025. To achieve this goal, the company has invested in wind and solar energy projects, as well as energy storage systems. This commitment has resulted in significant cost savings and has helped to reduce the company's carbon footprint.

Microsoft has also ma